Projekt z uczenia maszynowego - Jerzy Jankowski

Zadania: Celem drugiego projektu jest stworzenie dwóch klasyfikatorów danych krystalograficznych badanych w ramach analizy w języku R. Pierwszy klasyfikator powinien być nauczony na podstawie oryginalnego zbioru etykiet (res_name), a drugi na podstawie pogrupowanych etykiet (group_label), które dostarczy prowadzący.
Stworzone klasyfikatory będą oceniane na osobnym zbiorze danych, który dostarczy prowadzący. Ocena będzie oparta o uśrednioną po wszystkich klasach miarę recall (ang. macro-averaged recall).

Zadanie zrealizowano w języku Python z użyciem bibliotek Pandas i scikit-learn. Uzyskane estymacja miary recall dla klasyfikatorów mających być testowanych na zadanych danych testowych wyniosła odpowiednio: 0.41 i 0.44 dla danych oryginalnych i z podmienionymi etykietami.

In [3]:
"""Lista użytych bibliotek"""
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
import re
import cPickle

In [2]:
"""definicja klasy zawierającej wszystkie funkcjonalności pod postacią funkcji"""
class MyClassifierEngine:
    def __init__(self):
        pass

In [5]:
"""funkcje ładujące z plików data frame'y wejściowe"""
def loadFirstDF(self):
    return pd.read_csv("../input/all_summary.txt", sep=";", na_values=["nan"])
def loadResNames(self):
    return pd.read_csv("../input/grouped_res_name.txt", sep=",", na_values=["nan"])
def loadTestDataDF(self):
    return pd.read_csv("../input/test_data.txt", sep=",", na_values=["nan"])
MyClassifierEngine.loadFirstDF = loadFirstDF
MyClassifierEngine.loadResNames = loadResNames
MyClassifierEngine.loadTestDataDF = loadTestDataDF

In [7]:
""" funkcja odfiltrowująca kolumny:
  - z wartościami będącymi ciągami znaków, 
  - posiadającymi wyłącznie wartości Na, 
  - posiadające tylko jedną i tę samą wartość, co nie jest przydatne do trenowania, czy testowania klasyfikatora,
  - posiadające wartości pasujące do wzorców wyrażeń regularnych: '^part_0[1-9].*' i '^part_1.*', 
         ponieważ są mocno skorelowane z pozostawionymi kolumnami"""
def filterColumns(self, df):
    columnsWithStringValues = ["title", "chain_id", "res_id"]
    columnsWithOnlyNas = ["local_BAa", "local_NPa", "local_Ra", "local_RGa", "local_SRGa", "local_CCSa", "local_CCPa", "local_ZOa", "local_ZDa", "local_ZD_minus_a", "local_ZD_plus_a", "weight_col"]
    columnsWithSameValue = ["local_min", "fo_col", "fc_col", "grid_space", "solvent_radius", "solvent_opening_radius", "resolution_max_limit", "part_step_FoFc_std_min", "part_step_FoFc_std_max", "part_step_FoFc_std_step"]
    columnsToRemovePattern1 = r'^part_0[1-9].*'
    columnsToRemovePattern2 = r'^part_1.*'
    correctColumns = []
    for e in df.columns:
        if not( e in columnsWithStringValues or e in columnsWithOnlyNas or e in columnsWithSameValue or re.search(columnsToRemovePattern1, e) or re.search(columnsToRemovePattern2, e)):
            correctColumns.append(e)
    return df.loc[:, correctColumns]
MyClassifierEngine.filterColumns = filterColumns

In [8]:
"""funkcja odfiltrowująca kolumny dla danych testowych, w których nie ma 15 kolumn, które podnoszą jakość klasyfikacji. 
   Wykorzystuje funkcję filterColumns"""
def filterColumnsForTestData(self, df):
    df = self.filterColumns(df)
    columnsLackingInTestData = ["local_res_atom_count", "local_res_atom_non_h_count", "local_res_atom_non_h_occupancy_sum", "local_res_atom_non_h_electron_sum", "local_res_atom_non_h_electron_occupancy_sum", "local_res_atom_C_count", "local_res_atom_N_count", "local_res_atom_O_count", "local_res_atom_S_count", "dict_atom_non_h_count", "dict_atom_non_h_electron_sum", "dict_atom_C_count", "dict_atom_N_count", "dict_atom_O_count", "dict_atom_S_count", "local_volume", "local_electrons", "local_mean", "local_std"]
    correctColumns = []
    for e in df.columns:
        if not(e in columnsLackingInTestData):
            correctColumns.append(e)
    return df.loc[:, correctColumns]
MyClassifierEngine.filterColumnsForTestData = filterColumnsForTestData

In [10]:
"""Usunięcie wierszy posiadających wartość zmiennych res_name równą: 
   “DA”,“DC”,“DT”, “DU”, “DG”, “DI”,“UNK”, “UNX”, “UNL”, “PR”, “PD”, “Y1”, “EU”, “N”, “15P”, “UQ”, “PX4” lub “NAN”;"""
def filterRowsByResName(self, df):
    forbiddenResName = ["DA","DC","DT", "DU", "DG", "DI","UNK", "UNX", "UNL", "PR", "PD", "Y1", "EU", "N", "15P", "UQ", "PX4", "NAN"]
    return df[df["res_name"].apply(lambda x: not (x in forbiddenResName))]
MyClassifierEngine.filterRowsByResName = filterRowsByResName

In [11]:
"""W zbiorze danych uczących powinny zostać tylko unikatowe pary (pdb_code, res_name)"""
def filterRwosByDistinctPairs(self, df):
    e = []
    distinctSet = set([])
    for i in df.index:
        value = "{}_{}".format(df.loc[i,"pdb_code"],df.loc[i,"res_name"])
        if(value in distinctSet):
            e.append(False)
        else :
            distinctSet.add(value)
            e.append(True)
    return df[e]
MyClassifierEngine.filterRwosByDistinctPairs = filterRwosByDistinctPairs    

In [12]:
"""Pozostawienie wszystkich klas, których liczność wynosi co najmniej 5 przykładów; 
   klasy o mniejszej liczności nie mają być brane pod uwagę"""
def filterRowsByGroupSize(self, df, column="res_name", value = 5):
    f = df.groupby(column).size()
    return df[df[column].apply(lambda x: (pd.isnull(x) or (f[x] >= value)))]
MyClassifierEngine.filterRowsByGroupSize = filterRowsByGroupSize    

In [29]:
"""Nauczenie klasyfikatora, estymacja miary recall, wykorzystanie kodu optymalizującego parametry metodą grid search"""
def learnClfRandomForest(self, X, y, printBestParams = True, printClassificationReport = True):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)
    tuned_parameters = [{"max_depth": [10,50,None],
                 "criterion": ["gini", "entropy"],
                 "n_estimators": [2,10,30]}]
    scores = ['recall']

    for score in scores:
        clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5, scoring='%s_weighted' % score)
        clf.fit(X_train, y_train)
        if(printBestParams):
            print(clf.best_params_)

    y_true, y_pred = y_test, clf.predict(X_test)
    if(printClassificationReport):
        print(classification_report(y_true, y_pred))
    return clf
MyClassifierEngine.learnClfRandomForest = learnClfRandomForest  

In [15]:
"""Zapisywanie wyniku klasyfikacji do pliku. Wykorzystywane do utworzenia pliku z klasami przypisanymi do danych testowych
   zamieszczonych przez prowadzącego, w celu porównania przydzielonych klas z faktycznymi"""
def saveTestClassification(self, fileName, data):
    f = open('../{}.txt'.format(fileName),'w')
    j=1
    f.write('"","res_name"\n')
    for i in data:
        f.write('"{}","{}"\n'.format(j,i))
        j += 1
    f.close()
MyClassifierEngine.saveTestClassification = saveTestClassification  

In [16]:
"""Serializacja i deserializacja klasyfikatorów. Do serializacji wykorzystywano bibliotekę joblib, wg wymagań projektowych.
   Spostrzeżono jednak problemy z załadowaniem zserializowanego klasyfikatora typu RandomForest, zatem dodatkowo wykorzystano
   serializację i deserializację przy pomocy modułu cPickle."""
def exportClf(self, clf, fileName):
    joblib.dump(clf.best_estimator_, '{}.pkl'.format(fileName))
    with open('{}.pickle'.format(fileName), 'wb') as f:
        cPickle.dump(clf.best_estimator_, f)
def importClf(self, fileName):
    with open('{}.pickle'.format(fileName), 'rb') as f:
        return cPickle.load(f)
MyClassifierEngine.exportClf = exportClf 
MyClassifierEngine.importClf = importClf   

In [18]:
"""Metoda komponująca funkcję w jedną funkcjonalność - tworzenie klasyfikatora na podstawie danych podstawowych. Dane są ładowane, kolumny i wiersze są filtrowane.
   Po uzyskaniu wymaganych 11005 wierszy usuwane zostają wiersze zawierające wartości nan. 
   Takie wartości nie mogą być użyte do nauki klasyfikatora a zastąpienie ich wartością arbitralną lub pewną statystyką kolumny(średnią)
   mogłoby oszukać podczas trenowania klasyfikatora. Następnie ponownie usuwane są mało liczne grupy pod względem res_name.
   Utworzony klasyfikator zostaje poddany serializacji"""
def firstLearning(self):
    print("\n\nfirstLearning\n")
    df = self.loadFirstDF()
    df = self.filterColumns(df)
    df = self.filterRowsByResName(df)
    df = self.filterRwosByDistinctPairs(df)
    df = self.filterRowsByGroupSize(df)
    del df["pdb_code"]

    df = df.dropna(how="any")
    df = self.filterRowsByGroupSize(df)

    X = df.iloc[:,1:]
    y = df.iloc[:,0]
    clf = self.learnClfRandomForest(X,y)

    self.exportClf(clf, 'randomForest1')
    return clf
MyClassifierEngine.firstLearning = firstLearning  

In [19]:
"""Metoda komponująca funkcje, aby utworzyć klasyfikator ustawiający do danego data frame'a etykiety wierszy z pliku danego przez prowadzącego przedmiot.
   Od poprzedniej metody różni się sposobem uzyskania data frame'ów X oraz y wykorzystanych do nauczenia klasyfikatora.
   Indeksy wierszy zostają zsynchronizowane, ze starego data frame'a zostaje usunięta kolumna res_name i dodana kolumna res_name_group
   z data frame'a wczytanego z pliku."""
def secondLearning(self):
        print("\n\nsecondLearning\n")
        df = self.loadFirstDF()
        df = self.filterColumns(df)
        df = self.filterRowsByResName(df)
        df = self.filterRwosByDistinctPairs(df)
        df = self.filterRowsByGroupSize(df)
        del df["pdb_code"]

        del df["res_name"]
        dfy = self.loadResNames()
        df.index = dfy.index
        df = pd.concat([dfy.loc[:,"res_name_group"], df], axis=1)
        df = df.dropna(how="any")

        X = df.iloc[:,1:]
        y = df.iloc[:,0]
        clf = self.learnClfRandomForest(X,y)

        self.exportClf(clf, 'randomForest2')
        return clf
MyClassifierEngine.secondLearning = secondLearning  

In [31]:
"""Metoda tworząca klasyfikator wykorzystany do sklasyfikowania danych z pliku testowego, na etykietach oryginalnych. 
   Od poprzedniego klasyfikatora różni się wykorzystaniem innej metody do filtrowania kolumn data frame'a.
   filterColumnsForTestData zamiast filterColumns, ponieważ odfiltrowuje też ważne kolumny, których jednak nie ma w zbiorze danych testowych"""
def firstTestLearning(self):
    print("\n\nfirstTestLearning\n")
    df = self.loadFirstDF()
    df = self.filterColumnsForTestData(df)
    df = self.filterRowsByResName(df)
    df = self.filterRwosByDistinctPairs(df)
    df = self.filterRowsByGroupSize(df)
    del df["pdb_code"]

    df = df.dropna(how="any")
    df = self.filterRowsByGroupSize(df)

    X = df.iloc[:,1:]
    y = df.iloc[:,0]
    clf = self.learnClfRandomForest(X,y)

    self.exportClf(clf, 'randomForestFirstTest')
    return clf
MyClassifierEngine.firstTestLearning = firstTestLearning 

In [33]:
"""Metoda tworząca klasyfikator wykorzystany do sklasyfikowania danych z pliku testowego na etykietach pogrupowanych.
   Jak poprzednia metoda odfiltrowuje  ważne kolumny, których nie ma w zbiorze danych testowych"""
def secondTestLearning(self):
    print("\n\nsecondTestLearning\n")
    df = self.loadFirstDF()
    df = self.filterColumnsForTestData(df)
    df = self.filterRowsByResName(df)
    df = self.filterRwosByDistinctPairs(df)
    df = self.filterRowsByGroupSize(df)
    del df["pdb_code"]

    del df["res_name"]
    dfy = self.loadResNames()
    df.index = dfy.index
    df = pd.concat([dfy.loc[:,"res_name_group"], df], axis=1)
    df = df.dropna(how="any")

    X = df.iloc[:,1:]
    y = df.iloc[:,0]
    clf = self.learnClfRandomForest(X,y)

    self.exportClf(clf, 'randomForesSecondTest')
    return clf
MyClassifierEngine.secondTestLearning = secondTestLearning  

In [35]:
"""Metoda przeprowadzająca klasyfikację danych testowych. Wartości nan z pliku wejściowego zostają zamienione na 0. 
   Nie można pominąć wierszy zawierających takie wartości, gdyż dla każdego wiersza musi zostać przypisana klasa."""
def testFirstClassification(self):
    print("\n\ntestFirstClassification\n")
    df = self.loadTestDataDF()
    df = df.iloc[:,1:]
    df = self.filterColumnsForTestData(df)
    df = df.fillna(value=0)

    clf = self.importClf('randomForestFirstTest')
    print("**********************")
    result = clf.predict(df)
    self.saveTestClassification("outputFirst", result)

def testSecondClassification(self):
    print("\n\ntestClassification\n")
    df = self.loadTestDataDF()
    df = df.iloc[:,1:]
    df = self.filterColumnsForTestData(df)
    df = df.fillna(value=0)

    clf = self.importClf('randomForesSecondTest')
    print("**********************")
    result = clf.predict(df)
    self.saveTestClassification("outputSecond", result)
    
MyClassifierEngine.testFirstClassification = testFirstClassification 
MyClassifierEngine.testSecondClassification = testSecondClassification 

In [23]:
"""Utworzenie klasyfikatora na podstawie oryginalnego zbioru etykiet (res_name), z kolumnami nie występującymi w zbiorze testowym"""
mce1 = MyClassifierEngine()
mce1.firstLearning()
#mean recall: 0.87, dla danych testowych wydzielonych z oryginalnego zbioru danych
"""Utworzenie klasyfikatora na potstawie pogrupowanych etykiet, z kolumnami nie występującymi w zbiorze testowym"""
mce2 = MyClassifierEngine()
mce2.secondLearning()
#mean recall: 0.93, dla danych testowych wydzielonych z oryginalnego zbioru danych

In [36]:
"""Utworzenie klasyfikatora na podstawie oryginalnego zbioru etykiet, przeznaczony do testowania na zadanym zbiorze testowym"""
mce3 = MyClassifierEngine()
mce3.firstTestLearning()
#mean recall: 0.41, dla dodatkowych danych testowych 

In [36]:
"""Utworzenie klasyfikatora na podstawie oryginalnego zbioru etykiet, przeznaczony do testowania na zadanym zbiorze testowym"""
mce4 = MyClassifierEngine()
mce4.secondTestLearning()
#mean recall: 0.44, dla dodatkowych danych testowych 

KeyboardInterrupt: 

In [ ]:
"""Uruchomienie testów dla ostatnich dwóch klasyfikatorów na dostarczonych danych testowych"""
mceTestFirst = MyClassifierEngine()
mceTestFirst.testFirstClassification()

mceTestSecond = MyClassifierEngine()
mceTestSecond.testSecondClassification()